# Sesión de Trabajo 3: LIMPIEZA DE LA BASE DE DATOS
**Asignatura:** Ciència de Dades i Intel·ligència Artificial Aplicades a la Construcció i Estructures  
**Institución:** ETSEIB - UPC  

---

## Objetivos de la sesión
Antes de usar modelos estadísticos o de aprendizaje autónomo, siempre tenemos que limpiar los datos. Ningún modelo produce resultados significativos con datos sucios. En la sesión anterior, ya hicimos el primer paso para realizar la limpieza: detectar aquellas variables que requieren de correcciones. La sesión de trabajo actual se centra en corregir/rellenar/eliminar aquellos datos incorrectos/innecesarios/inconsistentes/nulos identificados perdiendo la mínima información posible. La sesión se estructura en diversas tareas.

* **S3.T1.** Tratamiento de los datos no relevantes

* **S3.T2.** Tratamiento de los datos inconsistentes

* **S3.T3.** Tratamiento de las observaciones nulas (missing data)

* **S3.T4.** Tratamiento de las observaciones atípicas (outliers)

No existe un orden ni estrategia establecida para encarar esta tarea. En general, se intenta empezar por aquellas acciones más simples y de mayor impacto y poco a poco se va hacia las acciones más específicas.
El objetivo es claro: obtener una BBDD relevante, consistente y sin observaciones nulas ni atípicas que sea lo más completa posible. Hay muchos caminos alternativos para alcanzarlo.

---


### S3.T0. Importar la base de datos
* Conectamos con Google Drive.
* Cargamos el archivo `BBDD_ST2.csv`.
* Optimizamos los tipos de datos (`float32` e `int32`) para no saturar la RAM de Colab.

⏳ *Tiempo de ejecución estimado: 1-2 minutos (dependiendo de la velocidad de conexión a Drive).*


In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Montar Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Ruta y Carga
ruta_drive = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST2.csv'
df = pd.read_csv(ruta_drive, low_memory=False)

# Optimización
float64_cols = df.select_dtypes(include='float64').columns.tolist()
df[float64_cols] = df[float64_cols].astype('float32')
int64_cols = df.select_dtypes(include='int64').columns.tolist()
df[int64_cols] = df[int64_cols].astype('int32')

print("Base de datos cargada y optimizada.")


---

### S3.T1. Tratamiento de datos NO RELEVANTES

* **S3.T1.1.** Identificar y Eliminar columnas no informativas debido a su alta repetitividad de sus observaciones.
* **S3.T1.2.** Identificar y Eliminar columnas no informativas debido a su nula relación con el fenómeno de estudio.
* **S3.T1.3.** Eliminar observaciones duplicadas. Esto se debería hacer al final de la limpieza de la BBDD.

---

* **S3.T1.1.** Identificar y Eliminar columnas no informativas debido a su alta repetitividad de sus observaciones.

    

>>EJEMPLO: Columnas no informativas por repetición de valores o NaNs

In [ ]:
df['SOLAR TERMICA'].value_counts(dropna=False) * 100 / len(df.index)

>>EJEMPLO: Variables con pocos datos pero valiosos. Antes de eliminar una variable, mejor analizar su distribución con atención y asegurar que no aporta información relevante para el fenómeno de estudio

In [ ]:
# 1. Preparación de datos
ind_ve = df[df['VEHICLE ELECTRIC'] == 'SI'].index
data_ve = df.loc[ind_ve, "Qualificació de consum d'energia primaria no renovable"].value_counts(normalize=True, dropna=False).sort_index()
data_global = df["Qualificació de consum d'energia primaria no renovable"].value_counts(normalize=True, dropna=False).sort_index()

# 2. Definir una paleta de colores fija para que las letras (A, B, C...) coincidan siempre
# Usamos un mapa de colores 'RdYlGn' (Rojo-Amarillo-Verde) invertido para eficiencia energética
colores = plt.cm.RdYlGn_r(np.linspace(0, 1, len(data_global)))

# 3. Crear la figura con 1 fila y 2 columnas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Subgrupo con Vehículo Eléctrico
data_ve.plot(
    kind="pie",
    ax=ax1,
    autopct='%1.1f%%',
    startangle=90,
    colors=colores,
    wedgeprops={'edgecolor': 'white'}
)
ax1.set_title("Edificios con VEHÍCULO ELÉCTRICO\n(Submuestreo)", fontweight='bold')
ax1.set_ylabel("") # Elimina el texto lateral innecesario

# Gráfico 2: Distribución Global
data_global.plot(
    kind="pie",
    ax=ax2,
    autopct='%1.1f%%',
    startangle=90,
    colors=colores,
    wedgeprops={'edgecolor': 'white'}
)
ax2.set_title("Distribución GLOBAL de la BBDD\n(Población Total)", fontweight='bold')
ax2.set_ylabel("")

>>*TAREA: Completa la definición de la variable "cols_baja_varianza" con las variables (columnas) que tienen falta de variabilidad (Repetitividad o NaNs).*

>>*Criterio: Columnas con >95% de valores idénticos o nulos.*

In [ ]:
cols_baja_varianza = [
    'SOLAR TERMICA',              # 100% "NO"
    'INFORME_INS_TECNICA_EDIFICI', # 99.83 % NaN
    # ... Añade aquí el resto de variables, poniendo al lado el motivo de su inclusión ...
]

>>Eliminar columnas no informativas por repetición de valores o NaNs

In [ ]:
df = df.drop(columns=cols_baja_varianza, errors='ignore')

* **S3.T1.2.** Identificar y Eliminar columnas no informativas debido a su nula relación con el fenómeno de estudio.

In [ ]:
# Lista de variables a eliminar por nula relación
cols_no_relacionadas = [
    'NUM_CAS',              # ID administrativo de la Generalitat
    'REFERENCIA CADASTRAL', # No influye en la física del edificio
    # --- Añade aquí el resto de variables detectadas ---
]

# Ejecución de la eliminación
df = df.drop(columns=cols_no_relacionadas, errors='ignore')

* **S3.T1.3.** Eliminar observaciones duplicadas. Esto se debería repetir al final de la limpieza de la BBDD.

In [ ]:
df = df.drop_duplicates()  # Eliminar observaciones duplicadas

---

### S3.T2. Tratamiento de datos INCONSISTENTES

* **S3.T2.1.** Uniformizar uso de las mayúsculas/minúsculas y corregir errores de escritura en variables categóricas.
* **S3.T2.2.** Errores de formato en direcciones, números, ...

---


TODO: Uniformizar uso de las mayÃºsculas/minÃºsculas y corregir errores de escritura en variables categÃ³ricas

EJEMPLO: Uniformizar mayÃºsculas/minÃºsculas

In [ ]:
df['NOM_PROVINCIA'].value_counts(dropna=False)  # DistribuciÃ³n de valores de NOM_PROVINCIA
df['NOM_PROVINCIA'] = df['NOM_PROVINCIA'].str.lower()  # Convertir a minÃºsculas

EJEMPLO: Corregir errores de escritura

In [ ]:
df['US_EDIFICI'].value_counts(dropna=False)  # DistribuciÃ³n de valores de US_EDIFICI
ind_typos = df[df['US_EDIFICI'] == 'Terciario'].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = Terciario
df.loc[ind_typos, 'US_EDIFICI'] = 'Terciari'  # Corregir US_EDIFICI = Terciario por Terciari

EJEMPLO: Eliminar observaciones con ciertas subcategorÃ­as errÃ³neas, pero que no tenemos criterio para corregir

In [ ]:
df['US_EDIFICI'].value_counts(dropna=False)  # DistribuciÃ³n de valores de US_EDIFICI
ind_typos = df[df["US_EDIFICI"] == "Habitatge"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = Habitatge
df.drop(ind_typos, axis=0)  # Eliminar observaciones con US_EDIFICI = Habitatge

EJEMPLO: Variable que el esfuerzo para corregir estos aspectos puede ser NO ASUMIBLE para el beneficio que aporta

In [ ]:
df['POBLACIO'].value_counts(dropna=False)  # DistribuciÃ³n de valores de POBLACIO. Vemos hay que muchisimos valores errÃ³neos

EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de NOM_PROVINCIA

In [ ]:
df['NOM_PROVINCIA'].value_counts(dropna=False)  # DistribuciÃ³n de valores de NOM_PROVINCIA
df['NOM_PROVINCIA'] = df['NOM_PROVINCIA'].str.lower()  # Convertir a minÃºsculas
ind_typos = df[df['NOM_PROVINCIA'] == '0'].index  # Identificar Ã­ndices de observaciones con NOM_PROVINCIA = 0
df = df.drop(ind_typos, axis=0)  # Eliminar observaciones con NOM_PROVINCIA = 0. No podemos corregir pues no tenemos criterio.
col_name = "NOM_PROVINCIA"  # Nombre de la variable a tratar
first_col = df.pop(col_name)  # Extraer la variable a tratar de df
df.insert(0, col_name, first_col)  # Insertar la variable a tratar en la primera posiciÃ³n de df

EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de US_EDIFICI

In [ ]:
df['US_EDIFICI'].value_counts(dropna=False)  # DistribuciÃ³n de valores de US_EDIFICI
df['US_EDIFICI'] = df['US_EDIFICI'].str.lower()  # Convertir a minÃºsculas
ind_typos = df[df['US_EDIFICI'] == 'terciario'].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = terciario
df.loc[ind_typos, 'US_EDIFICI'] = 'terciari'  # Corregir US_EDIFICI = terciario por terciari
ind_typos = df[df['US_EDIFICI'] == 'vivienda individual en bloque de viviendas'].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = vivienda individual en bloque de viviendas
df.loc[ind_typos, 'US_EDIFICI'] = "habitatge individual en bloc d'habitatges"  # Corregir US_EDIFICI = vivienda individual en bloque de viviendas por habitatge individual en bloc d'habitatges
ind_typos = df[df['US_EDIFICI'] == 'vivienda unifamiliar'].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = vivienda unifamiliar
df.loc[ind_typos, 'US_EDIFICI'] = "habitatge unifamiliar"  # Corregir US_EDIFICI = vivienda unifamiliar por habitatge unifamiliar
ind_typos = df[df['US_EDIFICI'] == "bloc d'habitatges"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = bloc d'habitatges
df.loc[ind_typos, 'US_EDIFICI'] = "bloc d'habitatges plurifamiliar"  # Corregir US_EDIFICI = bloc d'habitatges por bloc d'habitatges plurifamiliar
ind_typos = df[df['US_EDIFICI'] == "bloque de viviendas"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = bloque de viviendas
df.loc[ind_typos, 'US_EDIFICI'] = "bloc d'habitatges plurifamiliar"  # Corregir US_EDIFICI = bloque de viviendas por bloc d'habitatges plurifamiliar
ind_typos = df[df['US_EDIFICI'] == "habitatge plurifamiliar"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = habitatge plurifamiliar
df.loc[ind_typos, 'US_EDIFICI'] = "bloc d'habitatges plurifamiliar"  # Corregir US_EDIFICI = habitatge plurifamiliar por bloc d'habitatges plurifamiliar
ind_typos = df[df['US_EDIFICI'] == "habitatge unifamiliar aÃ¯llat"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = habitatge unifamiliar aÃ¯llat
df.loc[ind_typos, 'US_EDIFICI'] = "habitatge unifamiliar"  # Corregir US_EDIFICI = habitatge unifamiliar aÃ¯llat por habitatge unifamiliar
ind_typos = df[df['US_EDIFICI'] == "habitatge unifamiliar adossat"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = habitatge unifamiliar adossat
df.loc[ind_typos, 'US_EDIFICI'] = "habitatge unifamiliar"  # Corregir US_EDIFICI = habitatge unifamiliar adossat por habitatge unifamiliar
ind_typos = df[df['US_EDIFICI'] == "bloque de viviendas plurifamiliar"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = bloque de viviendas plurifamiliar
df.loc[ind_typos, 'US_EDIFICI'] = "bloc d'habitatges plurifamiliar"  # Corregir US_EDIFICI = bloque de viviendas plurifamiliar por bloc d'habitatges plurifamiliar
ind_typos = df[df['US_EDIFICI'] == "terciari: locals, oficines..."].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = terciari: locals, oficines...
df.loc[ind_typos, 'US_EDIFICI'] = "terciari"  # Corregir US_EDIFICI = terciari: locals, oficines... por terciari

In [ ]:
value_counts = df['US_EDIFICI'].value_counts(dropna=False)  # DistribuciÃ³n de valores de US_EDIFICI
to_remove = value_counts[value_counts <= 400].index  # Identificar Ã­ndices de US_EDIFICI con menos de 400 observaciones
df = df[~df.US_EDIFICI.isin(to_remove)]  # Eliminar observaciones con US_EDIFICI con menos de 400 observaciones

In [ ]:
col_name = "US_EDIFICI"  # Nombre de la variable a tratar
first_col = df.pop(col_name)  # Extraer la variable a tratar de df
df.insert(0, col_name, first_col)  # Insertar la variable a tratar en la primera posiciÃ³n de df

TAREA: Uniformiza el uso de mayÃºsculas/minÃºsculas y corrige los errores de escritura en variables categÃ³ricas.<br>
Para facilitar la tarea, se os empieza la limpieza de las variables.

In [ ]:
    # Limpieza COMARCA. Continuar la lista...

In [ ]:
df['COMARCA'].value_counts(dropna=False)

In [ ]:
ind_typos = df[df['COMARCA'] == "BarcelonÃ©s"].index
df.loc[ind_typos, 'COMARCA'] = "BarcelonÃ¨s"
ind_typos = df[df['COMARCA'] == "BarcelonÂ¿s"].index
df.loc[ind_typos, 'COMARCA'] = "BarcelonÃ¨s"
ind_typos = df[df['COMARCA'] == "VallÃ½s Oriental"].index
df.loc[ind_typos, 'COMARCA'] = "VallÃ¨s Oriental"

    # Limpieza de Eines de certificacio. Continuar la lista...

In [ ]:
index_null = df[df["Eina de certificacio"].isnull()].index
df = df.drop(index_null, axis=0)
ind_typos = df[df['Eina de certificacio'] == "CE3x"].index
df.loc[ind_typos, 'Eina de certificacio'] = "CE3X"
ind_typos = df[df['Eina de certificacio'] == "CALENER VYP"].index
df.loc[ind_typos, 'Eina de certificacio'] = "CALENER"

    # Limpieza Motiu de la certificacio. Continuar la lista...

In [ ]:
df['Motiu de la certificacio'] = df['Motiu de la certificacio'].str.lower()
ind_typos = df[df['Motiu de la certificacio'] == 'alquiler'].index
df.loc[ind_typos, 'Motiu de la certificacio'] = 'lloguer'
ind_typos = df[df['Motiu de la certificacio'] == 'compravenda'].index
df.loc[ind_typos, 'Motiu de la certificacio'] = 'compra o venda'

    # Limpieza Normativa construcciÃ³. Continuar la lista...

In [ ]:
df['Normativa construcciÃ³'] = df['Normativa construcciÃ³'].str.lower()
ind_typos = df[df['Normativa construcciÃ³'] == 'antes de 1979'].index
df.loc[ind_typos, 'Normativa construcciÃ³'] = 'abans de 1979'

    # Limpieza TITULACIO_TECNIC. Continuar la lista...

In [ ]:
df['TITULACIO_TECNIC'] = df['TITULACIO_TECNIC'].str.lower()
ind_typos = df[df['TITULACIO_TECNIC'] == 'arquitecto tÃ©cnico'].index
df.loc[ind_typos, 'TITULACIO_TECNIC'] = 'arquitecte tÃ¨cnic'

    # Limpieza PIS. Continuar la lista...(campos sugeridos: "SOT", "BX", "EN", "PR", "1", "2", "3", "4", "5-9", ">10", "AT", "SA")

In [ ]:
ind_typos = df[df['US_EDIFICI'] == "bloc d'habitatges plurifamiliar"].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = bloc d'habitatges plurifamiliar
df.loc[ind_typos, 'PIS'] = 'No aplica'  # En aquellos indices identificados, asignar PIS = No aplica, pues los bloques de habitajes no tienen pisos
ind_typos = df[df['US_EDIFICI'] == "habitatge unifamiliar"].index
df.loc[ind_typos, 'PIS'] = 'No aplica'
ind_typos = df[df['US_EDIFICI'] == "terciari"].index
df.loc[ind_typos, 'PIS'] = 'No aplica'
index_null = df[df["PIS"].isnull()].index  # Identificar Ã­ndices de observaciones con PIS = NaN
df = df.drop(index_null, axis=0)  # Eliminar observaciones con PIS = NaN
ind_typos = df[df['PIS'] == "-"].index  # Identificar Ã­ndices de observaciones con PIS = -
df = df.drop(ind_typos, axis=0)  # Eliminar observaciones con PIS = -
value_counts = df['PIS'].value_counts(dropna=False)
to_remove = value_counts[value_counts <= 100].index
df = df[~df.PIS.isin(to_remove)]

In [ ]:
ind_typos = df[df['PIS'] == '1Âº'].index
df.loc[ind_typos, 'PIS'] = '1'
ind_typos = df[df['PIS'] == '1r'].index
df.loc[ind_typos, 'PIS'] = '1'
ind_typos = df[df['PIS'] == '01'].index
df.loc[ind_typos, 'PIS'] = '1'
ind_typos = df[df['PIS'] == '1er'].index
df.loc[ind_typos, 'PIS'] = '1'

TODO: Errores de formato en direcciones, nÃºmeros, â€¦

EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de CODI_POSTAL

In [ ]:
df['CODI_POSTAL'].dtypes  # Comprobar el tipo de datos de la variable CODI_POSTAL

In [ ]:
index_null = df[df["CODI_POSTAL"].isnull()].index  # Identificar Ã­ndices de observaciones con CODI_POSTAL = NaN
df = df.drop(index_null, axis=0)  # Eliminar observaciones con CODI_POSTAL = NaN

In [ ]:
df['CODI_POSTAL'] = df['CODI_POSTAL'].astype(int)  # Convertir el tipo de datos de CODI_POSTAL a int
df['CODI_POSTAL'] = df['CODI_POSTAL'].astype(str)  # Convertir el tipo de datos de CODI_POSTAL a str
df['CODI_POSTAL'] = df['CODI_POSTAL'].str.zfill(5)  # Rellenar con ceros a la izquierda los valores de CODI_POSTAL hasta completar 5 dÃ­gitos

In [ ]:
col_name = "CODI_POSTAL"
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)

EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de Emissions de CO2 y Energia primÃ ria no renovable (la coma flotante hay que eliminarla)

In [ ]:
df["Emissions de CO2"] = [float(str(i).replace(",", "")) for i in df["Emissions de CO2"]]  # Eliminar la coma flotante de los valores de Emissions de CO2
df["Energia primÃ ria no renovable"] = [float(str(i).replace(",", "")) for i in df["Energia primÃ ria no renovable"]]  # Eliminar la coma flotante de los valores de Energia primÃ ria no renovable

EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de VALOR FINESTRES CTE y<br>
VALOR AILLAMENT CTE Para que se pandas detecte fÃ¡cilmente que se trata de una var. categÃ³rica.

In [ ]:
df['VALOR FINESTRES CTE'] = df['VALOR FINESTRES CTE'].astype(str)  # Convertir el tipo de datos de VALOR FINESTRES CTE a str
df['VALOR AILLAMENTS CTE'] = df['VALOR AILLAMENTS CTE'].astype(str)  # Convertir el tipo de datos de VALOR AILLAMENTS CTE a str


 S3.T3. Tratamiento de observaciones nulas (missing data)<br>
ïƒ˜ Trata las observaciones NaNs de tu base de datos con adoptando la alternativa mÃ¡s adiente en cada caso<br>


DiagnÃ³stico: Ver nÂº NaNs en cada columna

In [ ]:
Num_Nans = np.sum(df[df.columns].isnull())  # NÂº NaNs por columna

DiagnÃ³stico: Ver radiografÃ­a de distribuciÃ³n de NaNs en cada observaciÃ³n (fila)

In [ ]:
df2 = df.copy(deep=True)
for col in df2.columns:
    missing = df2[col].isnull()
    num_missing = np.sum(missing)
    if num_missing > 0:
        df2['{}_ismissing'.format(col)] = missing
ismissing_cols = [col for col in df2.columns if 'ismissing' in col]
df2['num_missing'] = df2[ismissing_cols].sum(axis=1)
missing_counts = df2['num_missing'].value_counts().reset_index()
missing_counts.columns = ['num_missing', 'count']
missing_counts.sort_values(by='num_missing', ascending=True, inplace=True)
missing_counts.plot.bar(x='num_missing', y='count', legend=False)
# Alternativa 1: Borrar filas con un nÃºmero elevado de NaNs

In [ ]:
ind_ausentes = df2[df2['num_missing'] > 6].index  # Identificar Ã­ndices de observaciones con num_missing > 6
df = df.drop(ind_ausentes, axis=0)  # Eliminar observaciones con num_missing > 6

Alternativa 2: Borrar filas puntuales

In [ ]:
df.drop([5], axis=0)  # Eliminar observaciÃ³n con Ã­ndice 5

Alternativa 3: Borrar todas las filas que tengan un NaN en una determinada columna. Apropiado si hay muy pocos NaNs en la columna

In [ ]:
df = df[df['NOM_PROVINCIA'].notna()]  # Eliminar observaciones con NaN en la columna NOM_PROVINCIA

Alternativa 4: Borrar columnas poco informativas que contengan observaciones nulas

In [ ]:
100*np.mean(df[df.columns[3]].isnull())  # Porcentaje de NaNs en la columna 3
df = df.drop(df.columns[3], axis=1)  # Eliminar columna 3

Alternativa 5: Asignar un valor a la observaciÃ³n nula. Distintos casos segÃºn variables contÃ­nuas o categÃ³ricas

In [ ]:
med = df['METRES_CADASTRE'].median()  # Calcular la mediana de la variable METRES_CADASTRE
df['METRES_CADASTRE'] = df['METRES_CADASTRE'].fillna(med)  # Sustituimos NaNs por la mediana

In [ ]:
top = df['SISTEMA BIOMASSA'].describe()['top']  # Calcular el valor mÃ¡s frecuente de la variable SISTEMA BIOMASSA
df['SISTEMA BIOMASSA'] = df['SISTEMA BIOMASSA'].fillna(top)  # Sustituimos NaNs por el campo mÃ¡s comÃºn

In [ ]:
df['XARXA DISTRICTE'] = df['XARXA DISTRICTE'].fillna('MISSING')  # Sustituimos NaNs una categorÃ­a nueva

TAREA: Trata las observaciones NaNs de tu base de datos con adoptando la alternativa mÃ¡s adiente en cada caso.<br>
AÃ±ade comentarios explicando tu decisiÃ³n.

EJEMPLO Limpieza de ZONA CLIMATICA.

In [ ]:
num_missing = np.sum(df["ZONA CLIMATICA"].isnull())
df = df[df['ZONA CLIMATICA'].notna()]  # CÃ³mo solo tengo 46 NaNs, elimino las observaciones asociadas

In [ ]:
col_name = "ZONA CLIMATICA"
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)


 S3.T4. Tratamiento de outliers<br>
ïƒ˜ Elimina los outliers de tu base de datos de las variables continuas<br>


EJEMPLO: IdentificaciÃ³n de outliers en variables contÃ­nuas

In [ ]:
df['METRES_CADASTRE'].describe()  # DescripciÃ³n de la variable METRES_CADASTRE

In [ ]:
import seaborn as sns  # GrÃ¡fica de caja y bigotes
sns.kdeplot(data=[df['METRES_CADASTRE']])  # GrÃ¡fico de densidad de la variable METRES_CADASTRE

EJEMPLO: Eliminar observaciones con outliers en variables contÃ­nuas

In [ ]:
ind_out = df[df['METRES_CADASTRE'] < 5].index  # Identificar Ã­ndices de observaciones con METRES_CADASTRE < 5 mÂ²
df = df.drop(ind_out, axis=0)  # Eliminar observaciones con METRES_CADASTRE < 5 mÂ²
ind_outliers = df[df['METRES_CADASTRE'] > 500].index  # Identificar Ã­ndices de observaciones con METRES_CADASTRE > 500 mÂ²
df = df.drop(ind_outliers, axis=0)  # Eliminar observaciones con METRES_CADASTRE > 500 mÂ²
col_name = "METRES_CADASTRE"
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)

EJEMPLO: IdentificaciÃ³n de outliers en variables categÃ³ricas

In [ ]:
df['US_EDIFICI'].value_counts().plot.bar()  # GrÃ¡fico de barras de la variable US_EDIFICI

EJEMPLO: Eliminar observaciones con outliers en variables categÃ³ricas

In [ ]:
ind_out = df[df['US_EDIFICI'] == 'MULTIUSOS'].index  # Identificar Ã­ndices de observaciones con US_EDIFICI = 'MULTIUSOS'
df = df.drop(ind_out, axis=0)  # Eliminar observaciones con US_EDIFICI = 'MULTIUSOS'

TAREA: Elimina los outliers de tu base de datos (compacitat, Emissions de CO2, Energia primÃ ria no renovable)

TODO: Eliminar observaciones duplicadas.

In [ ]:
df = df.drop_duplicates()  # Eliminar observaciones duplicadas

TODO: Guardar BBDD

In [ ]:
df.to_csv(r'G:\Mi unidad\Docencia\Repo_projectes1\BBDD_ST3.csv', index=False)  # Guardar BBDD


TAREAS FINALES SESIÃ“N TRABAJO 3<br>
DESCRIPCIÃ“N DE LA TAREA: Contesta a las siguientes preguntas sobre tu nueva base de datos<br>
ïƒ˜NÃºmero de filas antes y despuÃ©s de la limpieza?<br>
ïƒ˜NÃºmero de columnas antes y despuÃ©s de la limpieza?<br>
ïƒ˜Nombre de las columnas que usarÃ¡s como posibles INPUTS (Entradas) en el proceso de certificaciÃ³n?<br>
ïƒ˜Nombre de las columnas que usarÃ¡s como posibles OUTPUTS (Salidas) del proceso de certificaciÃ³n?<br>
ïƒ˜En las variables categÃ³ricas, indicad el nÃºmero de campos posibles antes y despuÃ©s de la limpieza<br>
ïƒ˜En las variables continuas, indicad la distribuciÃ³n inicial y final de los valores numÃ©ricos (con un kdeplot)<br>
